<a href="https://colab.research.google.com/github/Rahafhosari/DataScience2024-2025/blob/master/classification_metrics_core.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Classification Metrics

Name: Rahaf Hosari

### Mount Drive

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


### Imports

In [14]:
from sklearn.metrics import accuracy_score
from sklearn.metrics import recall_score
from sklearn.metrics import precision_score
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.metrics import ConfusionMatrixDisplay
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import make_pipeline
from sklearn.tree import DecisionTreeClassifier
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np

### Load Data

1) Check for class balance with value_counts

In [7]:
path = '/content/drive/MyDrive/AXSOSACADEMY/02-IntroML/Week08/Classification Metrics/stroke.csv'
df = pd.read_csv(path)

In [8]:
df.info()
df.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1137 entries, 0 to 1136
Data columns (total 12 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   id                 1137 non-null   int64  
 1   gender             1137 non-null   object 
 2   age                1137 non-null   object 
 3   hypertension       1137 non-null   int64  
 4   heart_disease      1137 non-null   int64  
 5   ever_married       1137 non-null   object 
 6   work_type          1137 non-null   object 
 7   Residence_type     1137 non-null   object 
 8   avg_glucose_level  1137 non-null   float64
 9   bmi                1085 non-null   float64
 10  smoking_status     1137 non-null   object 
 11  stroke             1137 non-null   int64  
dtypes: float64(2), int64(4), object(6)
memory usage: 106.7+ KB


,id,gender,age,hypertension,heart_disease,ever_married,work_type,Residence_type,avg_glucose_level,bmi,smoking_status,stroke
0,1192,Female,31,0,0,No,Govt_job,Rural,70.66,27.2,never smoked,0
1,77,Female,13,0,0,No,children,Rural,85.81,18.6,Unknown,0
2,59200,Male,18,0,0,No,Private,Urban,60.56,33.0,never smoked,0
3,24905,Female,65,0,0,Yes,Private,Urban,205.77,46.0,formerly smoked,1
4,24257,Male,4,0,0,No,children,Rural,90.42,16.2,Unknown,0


## 1.Check for class balance with value_counts

In [9]:
df['stroke'].value_counts()

,count
stroke,
0,1000
1,137


The specimen is unbalanced, indicating classes are not equally repesented. People with stroke are less than people without in this dataset.

## 2.Prepare a column transformer for preprocessing

### Train Test Split

In [11]:
# Separate X,y and train test split
target = 'stroke'
X = df.drop(columns=target)
y = df[target]
X_train, X_test, y_train, y_test = train_test_split(X,y, random_state=42)
# Checking the class balance
y_train.value_counts(normalize=True).sort_index()

,proportion
stroke,
0,0.879108
1,0.120892


### Standard Scaler

In [12]:
scaler = StandardScaler()

## 3. Build and fit a model pipeline with the preprocessor and a default decision tree classifer (use a random_state of 42)

In [17]:
clf_pipe = make_pipeline(scaler,DecisionTreeClassifier(random_state=42))
clf_pipe

Pipeline(steps=[('standardscaler', StandardScaler()),
                ('decisiontreeclassifier',
                 DecisionTreeClassifier(random_state=42))])

### Fit Data

In [18]:
clf_pipe.fit(X_train, y_train)
y_pred_test = clf_pipe.predict(X_test)

ValueError: could not convert string to float: 'Female'